In [9]:
import tomographer
#import logging
#tomographer.cxxlogger.level = logging.NOTSET # slow things down a bit, to test for progress

In [10]:
# Give us a nice progress bar
from ipywidgets import IntProgress, HTML, VBox, HBox
from IPython.display import display  

In [13]:
import numpy as np

Nm = np.array([ 490, 510, 494, 506, 980, 20 ])

Emn = [
    # X direction
    np.array([[1, 1],
              [1, 1]]),
    np.array([[1, -1],
              [-1, 1]]),
    # Y direction
    np.array([[1, -1j],
              [1j, 1]]),
    np.array([[1, 1j],
              [-1j, 1]]),
    # Z direction
    np.array([[1, 0],
              [0, 0]]),
    np.array([[0, 0],
              [0, 1]]),
    ]

rho_ref = np.array([[1,0],[0,0]])

# give ourselves a nice progress bar
class NiceProgressBar(object):
    def __init__(self, label_msg_html="Progress: "):
        self.progress_widget = IntProgress(min=0, max=100, value=0)
        self.label = HTML("<span style=\"padding-right: 1em;\">"+label_msg_html+"</span>")
        self.labelval = HTML()
        self.box = HBox(children=[self.label, self.progress_widget, self.labelval])
        display(self.box)
        super(NiceProgressBar, self).__init__()
    def __enter__(self):
        self.progress_widget.value = 0
        self.progress_widget.bar_style = 'info'
        return self
    def progress(self, **kwargs):
        if 'fraction_done' in kwargs:
            pcvalue = int(kwargs['fraction_done']*100)
        elif 'percent_done' in kwargs:
            pcvalue = int(kwargs['percent_done'])
        else:
            raise ArgumentError("No fraction_done nor percent_done argument given")
        self.progress_widget.value = pcvalue
        self.labelval.value = "<span style=\"font-weight: bold; padding-left: 1em;\">{}% done</span>".format(pcvalue)
    def __exit__(self, etype, evalue, etraceback):
        self.progress_widget.value = 100
        if etype:
            self.progress_widget.bar_style = 'danger'
        else:
            self.progress_widget.bar_style = 'success'

r = None
with NiceProgressBar("Running random walks for the tomography procedure ... ") as prg:
    def progress_fn(r):
        prg.progress(fraction_done=r.total_fraction_done)
        #print("{:.2%}: {} + ({}) ".format(r.total_fraction_done, r.num_completed,
        #                                ",".join(["{:.2%}".format(w.fraction_done) for w in r.workers if w]) ))
    r = tomographer.tomorun.tomorun(dim=2, Nm=Nm, Emn=Emn,
                                    hist_params=tomographer.UniformBinsHistogramParams(0.9,1,50),
                                    mhrw_params=tomographer.MHRWParams(0.03,35,500,8192),
                                    fig_of_merit="obs-value",
                                    num_repeats=12,
                                    observable=rho_ref,
                                    progress_fn=progress_fn)

print(repr(r))

print(r['final_histogram'].prettyPrint(100))

print(r['final_report'])


{'simple_final_histogram': <tomographer.AveragedSimpleRealHistogram object at 0x7f2a4c0a2030>, 'final_report': '\n                                        Final Report of Runs                                        \n----------------------------------------------------------------------------------------------------\n\tstep size       = 0.03\n\t# iter. / sweep = 35\n\t# therm. sweeps = 500\n\t# run sweeps    = 8192\n# 0: 0.9|                             ......-+o%##%o-...   |1                  [accept ratio = 0.20]\n    error bars: 50 converged / 0 maybe (0 isolated) / 0 not converged\n# 1: 0.9|                           . ......-+x%##%o-...   |1                  [accept ratio = 0.20]\n    error bars: 50 converged / 0 maybe (0 isolated) / 0 not converged\n# 2: 0.9|                           . ......-+o%##%o-..    |1                  [accept ratio = 0.20]\n    error bars: 50 converged / 0 maybe (0 isolated) / 0 not converged\n# 3: 0.9|                            . .....-+o%##%o-...   |1 

In [8]:
print(r['final_histogram'].params.min, r['final_histogram'].params.max, r['final_histogram'].params.num_bins)
print(r['final_histogram'].bins)

0.9 1.0 50
[[  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  1.01725260e-05]
 [  2.03450521e-05]
 [  5.08626302e-05]
 [  2.13623047e-04]
 [  4.37418620e-04]
 [  1.52587891e-03]
 [  3.37727865e-03]
 [  8.43302409e-03]
 [  1.83614095e-02]
 [  3.58581543e-02]
 [  6.42395020e-02]
 [  1.04309082e-01]
 [  1.46392822e-01]
 [  1.74296061e-01]
 [  1.72882080e-01]
 [  1.37654622e-01]
 [  8.37605794e-02]
 [  3.64481608e-02]
 [  9.95890299e-03]
 [  1.68863932e-03]
 [  8.13802083e-05]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.000